In [3]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.5 MB/s eta 0:00:00


In [4]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 7.9 MB/s eta 0:00:00


In [5]:
!pip install transformers

In [6]:
import numpy as np
import pandas as pd

import re
import string # for text cleaning
import contractions # for expanding short form words
from tqdm import tqdm
tqdm.pandas(desc="Progress Bar")

import torch
#from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel,AutoModel,AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
import pandas as pd
tr='/kaggle/input/rikruta/training_data.csv'
cv='/kaggle/input/rikruta/cv_data.csv'
jd=pd.read_csv(tr)
cv=pd.read_csv(cv)
cv

,Education,Skills,Experience,Projects,Certifications,Contact,Filename
0,Hosted television show that focused on campus ...,"Skills academic, ads, advertising, banners, br...",Experience 09/2013 to Current Media Activities...,NaN,NaN,Designated as the contact for area journalists...,10005171.pdf
1,"Education and Training Associates , Applied Sc...",skills and continue to gain experience in the ...,My main objective in seeking employment with T...,NaN,Responsible for testing various seat structure...,NaN,10030015.pdf
2,"Education Glendale College of Law City , State...","Computer Skills Proficient: Epic, Word, Lotus,...",Experience 07/2002 - CurrentCompany Name - Cit...,NaN,License LCS 18224Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â...,NaN,10062724.pdf
3,Maintained partnerships with external customer...,Skills,Experience 10/2008 to Current Chief Executive ...,Project Management Sales Operations Relationsh...,NaN,NaN,10076271.pdf
4,Giving and participating in training and educa...,Versatile Systems Administrator possessing sup...,VMWare experience Disaster recovery Experience...,Collaborating on project plan for Office 365 m...,Certifications CompTIA Network+ - 2014,Configuring and supporting Blackberry devices ...,10089434.pdf
5,Education General Courses : Monterey Peninsula...,SkilledÂ Maintenance MechanicÂ Technician with...,years of Law Enforcement experience with the U...,NaN,"Writes DSN's for over shipment, wrong parts, a...",NaN,10219099.pdf
6,Education Associate of Science : Business Admi...,"Skills Loans, Mortgage, Documentation, Lending...",large spectrum of experience in the financial ...,NaN,NaN,NaN,3547447.pdf
7,Education: MBA in Finance | Harvard Business S...,"Skills: Financial Modeling, Budgeting, Forecas...",Financial Analyst with 6 years of experience i...,NaN,NaN,Contact Information: Email: anonymous_finance@...,87.docx
8,Education: MSc in Machine Learning | Carnegie ...,"Skills: NLP: Named Entity Recognition (NER), T...",Natural Language Processing Engineer with 4+ y...,NaN,NaN,Contact Information: Email: anonymous_nlp_engi...,;ko.docx
9,"EDUCATION Federal University of Technology, Mi...","SKILLS Languages: HTML,CSS,JavaScript,React,Bo...",EXPERIENCE Front-end Developer Sep. 2022–Prese...,PROJECTS,•CERTIFICATION • Zuri/I4GFrontendInternship: A...,Phone-Alt+2348137290905 | Envelope adetomiwaab...,Abdulrasheed_Abdulsalam.pdf


In [19]:
cv.Education

0     Hosted television show that focused on campus ...
1     Education and Training Associates , Applied Sc...
2     Education Glendale College of Law City , State...
3     Maintained partnerships with external customer...
4     Giving and participating in training and educa...
5     Education General Courses : Monterey Peninsula...
6     Education Associate of Science : Business Admi...
7     Education: MBA in Finance | Harvard Business S...
8     Education: MSc in Machine Learning | Carnegie ...
9     EDUCATION Federal University of Technology, Mi...
10    EDUCATION Federal University of Technology, Mi...
11    Education: BSc in Accounting | University of M...
12    EDUCATION Federal University of Technology, Mi...
13    Education: MSc in Artificial Intelligence | MI...
14    EDUCATION ▪ 2023 Bachelor of Science (Statisti...
15    1 Post Graduate School,Computer 2021-In view D...
16    Education: PhD in Machine Learning | Universit...
17    Education: MSc in Data Science | Universit

In [3]:
cv.loc[0]

Education         Hosted television show that focused on campus ...
Skills            Skills academic, ads, advertising, banners, br...
Experience        Experience 09/2013 to Current Media Activities...
Projects                                                        NaN
Certifications                                                  NaN
Contact           Designated as the contact for area journalists...
Filename                                               10005171.pdf
Name: 0, dtype: object

In [9]:
import numpy as np
import pandas as pd

import re
import string # for text cleaning
import contractions # for expanding short form words
from tqdm import tqdm
tqdm.pandas(desc="Progress Bar")

import torch
#from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel,AutoModel,AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
def text_cleaning(text:str) -> str:
    if pd.isnull(text):
        return
    # lower-case everything
    text = text.lower().strip()
    # For removing puctuations
    translator = str.maketrans('', '', string.punctuation)
    # expand all the short-form words
    text = contractions.fix(text)
    # remove any special chars
    text = re.sub(r'http\S+|www\S+|https\S+', '', text) # Remove URLs
    text = re.sub(r'\S+@\S+', '', text) # Remove emails
    text = re.sub(r'\b\d{1,3}[-./]?\d{1,3}[-./]?\d{1,4}\b', '', text) # Remove phone numbers
    text = text.translate(translator) # Remove puctuations
    text = re.sub(r'[^a-zA-Z]', ' ', text) # Remove other non-alphanumeric characters
    return text.strip()

In [23]:
cv.isna().sum()

Education          0
Skills             2
Experience         1
Projects          32
Certifications    36
Contact            5
Filename           0
dtype: int64

In [11]:
# Filling the null values in Skills & Education with Empty String before concatinating them
cv = cv.fillna(value='')

# Let's stitch together Skills & Education, similar to given in job description.
cv['CV'] = cv['Skills'] + ' ' + cv['Education']

# Doing text cleaning
cv['CV'] = cv['CV'].progress_apply(text_cleaning)

Progress Bar: 100%|██████████| 42/42 [00:00<00:00, 1427.13it/s]


In [12]:
# Sample job descriptions
job_descriptions = jd['job_description'].apply(text_cleaning)[:10].to_list() 
# Sample resumes (replace with your extracted resume data)
resumes = cv['CV'].to_list()

In [13]:
s_tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
s_model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [29]:
%%time
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

resume_embeddings = []
for resume in resumes:
    tokens = s_tokenizer(resume, padding=True, truncation=True, max_length=24, return_tensors='pt')
    with torch.no_grad():  # Compute token embeddings
        s_model_output = s_model(**tokens)
    # Perform mean pooling to get the sentence embedding
    pooled_embedding = mean_pooling(s_model_output, tokens['attention_mask'])
    resume_embeddings.append(pooled_embedding.numpy()[0])  # Flatten to 1D and add to list

jd_embeddings = []
for description in job_descriptions:
    tokens = tokenizer(description, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():  # Compute token embeddings
        s_model_output = s_model(**tokens)
    # Perform mean pooling to get the sentence embedding
    pooled_embedding = mean_pooling(s_model_output, tokens['attention_mask'])
    jd_embeddings.append(pooled_embedding.numpy()[0])  # Flatten to 1D and add to list


UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


In [30]:
%%time
# Tokenize and embed job descriptions
jd_embeddings = []
for description in job_descriptions:
    tokens = tokenizer(description, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        output = model(**tokens)
    # Apply mean pooling to get the sentence embedding
    pooled_embedding = mean_pooling(output, tokens['attention_mask'])
    jd_embeddings.append(pooled_embedding.numpy()[0])  # Flatten to 1D and add to list

# Tokenize and embed resumes
resume_embeddings = []
for resume in resumes:
    tokens = tokenizer(resume, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        output = model(**tokens)
    # Apply mean pooling to get the sentence embedding
    pooled_embedding = mean_pooling(output, tokens['attention_mask'])
    resume_embeddings.append(pooled_embedding.numpy()[0])  # Flatten to 1D and add to list


CPU times: user 18.6 s, sys: 22.4 ms, total: 18.6 s
Wall time: 9.43 s


In [3]:
# Calculate cosine similarity between job descriptions and resumes
similarity_scores = cosine_similarity(jd_embeddings, resume_embeddings)

# Rank candidates for each job description based on similarity scores
num_top_candidates = 5
top_candidates = []

for i, job_description in enumerate(job_descriptions):
    # Convert similarity scores to percentages
    candidates_with_scores = [(index, score * 100) for index, score in enumerate(similarity_scores[i])]
    candidates_with_scores.sort(key=lambda x: x[1], reverse=True)
    top_candidates_for_job = candidates_with_scores[:num_top_candidates]
    top_candidates.append(top_candidates_for_job)

# Print the top candidates for each job description with percentage match
for i, job_description in enumerate(job_descriptions):
    print(f"Top candidates for JD {i+1} - Position: {jd['position_title'][i]}")
    for candidate_index, score in top_candidates[i]:
        print(f"  Candidate {candidate_index + 1} - Match Percentage: {score:.2f}% - {cv['Filename'][candidate_index]}.pdf")
    print()


NameError: name 'cosine_similarity' is not defined